In [5]:
import pandas as pd

In [25]:
compustat = pd.read_csv(r"C:\Etudes\EPFL\1ère\MA2\ML\PROJECT\data_raw\Predictors\CompFirmCharac.csv\CompFirmCharac.csv")

C:\Users\rvent\AppData\Local\Temp\ipykernel_16560\3602216866.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  compustat = pd.read_csv(r"C:\Etudes\EPFL\1ère\MA2\ML\PROJECT\data_raw\Predictors\CompFirmCharac.csv\CompFirmCharac.csv")


In [31]:
compustat.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xoptepsqpy,xoptepsy,xoptqpy,xopty,xrdy,xsgay,exchg,cik,costat,fic
0,1000,1966-03-31,1966,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
1,1000,1966-06-30,1966,2.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
2,1000,1966-09-30,1966,3.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
3,1000,1966-12-31,1966,4.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
4,1000,1967-03-31,1967,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA


In [ ]:
# Possibility to check the 10Q or 10K fillinf date?
#Eg If I have a datadate 1966.03.31, it  means that this financial statement cover the quarter that ended 
# at this date
#So the associated information will be avaialable after a specfic lag. 


In [75]:
import pandas as pd

def identify_string_and_mixed_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a DataFrame listing each column that contains non-numeric (object) or mixed-type values,
    along with its pandas dtype and a sample of Python types observed.
    """
    result = []
    for col in df.columns:
        if df[col].dtype == 'object':
            # collect up to 5 unique Python types in this column
            sample_types = df[col].dropna().map(type).unique()[:5].tolist()
            result.append({
                'column': col,
                'pandas_dtype': df[col].dtype,
                'sample_types': sample_types
            })
    return pd.DataFrame(result)

# Usage:
mixed_df = identify_string_and_mixed_columns(compustat)
print(mixed_df.to_string(index=False))



  column pandas_dtype                                    sample_types
     tic       object                                 [<class 'str'>]
   cusip       object [<class 'str'>, <class 'int'>, <class 'float'>]
    conm       object                                 [<class 'str'>]
  curcdq       object                                 [<class 'str'>]
datacqtr       object                                 [<class 'str'>]
datafqtr       object                                 [<class 'str'>]
  costat       object                                 [<class 'str'>]
     fic       object                                 [<class 'str'>]


In [78]:
# On récupère les dtypes sous forme de Series :
dtypes = compustat.dtypes

# On filtre pour ne garder que ceux qui ne sont pas numériques
mask_non_num = ~dtypes.apply(pd.api.types.is_numeric_dtype)
df_non_num = (
    dtypes[mask_non_num]
    .reset_index()
    .rename(columns={'index': 'column', 0: 'pandas_dtype'})
)

print(df_non_num)


     column    pandas_dtype
0  datadate  datetime64[ns]
1       tic          object
2     cusip          object
3      conm          object
4    curcdq          object
5  datacqtr          object
6  datafqtr          object
7    costat          object
8       fic          object


In [79]:
compustat['conm'].head()

0    A & E PLASTIK PAK INC
1    A & E PLASTIK PAK INC
2    A & E PLASTIK PAK INC
3    A & E PLASTIK PAK INC
4    A & E PLASTIK PAK INC
Name: conm, dtype: object

In [ ]:
#Preliminary cleaning:

#1) Keep only industrial firm (avoid distortion of classic ratio and consistency with benchmark studies as JKP)
compustat = compustat[compustat['indfmt'] == 'INDL']

#2) Keep only consolidated financial statement to avoid double counting
compustat = compustat[compustat['consol'] == 'C']

#3) Keep only GAAP Standard format: Ensure consistency
compustat = compustat[compustat['datafmt'] == 'STD']

#4) Check if there are any duplicate  pair (gvkey, datadate):
#A voir si on drop les duplicate keys -> Ca fou la merde dans le ML sinon on a quand meme 1550 lignes avec la clef dupliquée.

#5) Lag the features A VOIR SI LE DATASET EST DEJA LAGGE EN CE SENS (EN ATTENTE DU GIUS SUR ED) SI PAS LAGGUE DECALLE DE 3 MOIS

# 6) Convert data to datetime:
compustat['datadate'] = pd.to_datetime(compustat['datadate'])

#7) Drop raw identifier columns:
compustat = compustat.drop(columns=['tic','cusip','conm'])

#8) Keep only active firms


In [82]:
# Afficher le nombre d’occurrences pour chaque valeur de 'costat'
print(compustat['costat'].value_counts(dropna=False))



costat
I    1208184
A     843830
Name: count, dtype: int64


In [ ]:
#Build a function where we check wheter a collumn is always the same for all the dataset

def print_constant_columns(df):
    """
    Print all columns of the DataFrame that contain a single unique value (including NaN).
    
    For each such column, outputs the column name and the constant value it holds.
    """
    
    const_series = df.loc[:, df.nunique(dropna=False) == 1].iloc[0]
    print(const_series.to_string())

# Usage:
print_constant_columns(compustat)


indfmt     INDL
consol        C
popsrc        D
datafmt     STD


In [ ]:
#WORK ON COLLUMNS

#8) Drop all columns that have exactly one unique value
compustat = compustat.drop(
    [col for col in compustat.columns if compustat[col].nunique(dropna=False) == 1],
    axis=1
)

In [55]:
def count_high_nan_columns(df, threshold):
    """
    Count how many columns in df have more than `threshold` fraction of missing values.
    
    Parameters
    ----------
    df : DataFrame
        The DataFrame to inspect.
    threshold : float, default 0.7
        Fraction of rows that must be NaN to count the column.
    
    Returns
    -------
    int
        Number of columns with > threshold fraction of NaNs.
    """
    return (df.isna().mean() > threshold).sum()

count_high_nan_columns(compustat, 0.7)

np.int64(159)

In [64]:

def count_nan_columns_by_threshold(df: pd.DataFrame, thresholds=None) -> dict:
    """
    Count how many columns in `df` have a proportion of NaNs strictly greater than each threshold.

    """
    if thresholds is None:
        thresholds = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.60, 0.50, 0.40, 0.30,0.20,0.10]
    na_frac = df.isna().mean()
    return {thr: int((na_frac >= thr).sum()) for thr in thresholds}

# Usage example:
counts = count_nan_columns_by_threshold(compustat)
print(counts)


{0.95: 107, 0.9: 120, 0.85: 132, 0.8: 150, 0.75: 156, 0.7: 159, 0.6: 174, 0.5: 185, 0.4: 205, 0.3: 231, 0.2: 237, 0.1: 238}


In [65]:

def drop_nan_columns_by_threshold(df: pd.DataFrame, threshold: float) -> DataFrame:
    """
    Drop columns in `df` where the fraction of NaNs is greater than or equal to `threshold`.

    """
    na_frac = df.isna().mean()
    cols_to_keep = na_frac[na_frac < threshold].index
    return df.loc[:, cols_to_keep]

drop_nan_columns_by_threshold(compustat,0.7)

,gvkey,datadate,fyearq,fqtr,fyr,tic,cusip,conm,curcdq,datacqtr,...,xidocy,xidoy,xinty,xiy,xopry,xsgay,exchg,cik,costat,fic
0,1000,1966-03-31,1966,1.0,12,AE.2,000032102,A & E PLASTIK PAK INC,USD,1966Q1,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
1,1000,1966-06-30,1966,2.0,12,AE.2,000032102,A & E PLASTIK PAK INC,USD,1966Q2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
2,1000,1966-09-30,1966,3.0,12,AE.2,000032102,A & E PLASTIK PAK INC,USD,1966Q3,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
3,1000,1966-12-31,1966,4.0,12,AE.2,000032102,A & E PLASTIK PAK INC,USD,1966Q4,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
4,1000,1967-03-31,1967,1.0,12,AE.2,000032102,A & E PLASTIK PAK INC,USD,1967Q1,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052009,356687,2023-09-30,2023,3.0,12,SASKF,046824108,ATHA ENERGY CORP,CAD,2023Q3,...,0.0,0.0,0.000,0.0,8.866,8.866,19.0,NaN,A,CAN
2052010,356687,2023-12-31,2023,4.0,12,SASKF,046824108,ATHA ENERGY CORP,CAD,2023Q4,...,0.0,0.0,0.000,0.0,15.103,15.103,19.0,NaN,A,CAN
2052011,356687,2024-03-31,2024,1.0,12,SASKF,046824108,ATHA ENERGY CORP,CAD,2024Q1,...,0.0,0.0,0.001,0.0,4.302,4.302,19.0,NaN,A,CAN
2052012,356687,2024-06-30,2024,2.0,12,SASKF,046824108,ATHA ENERGY CORP,CAD,2024Q2,...,0.0,0.0,0.008,0.0,8.307,8.307,19.0,NaN,A,CAN


In [66]:
def count_nan_rows_by_threshold(df: pd.DataFrame, thresholds=None) -> dict:
    """
    Count how many rows in `df` have a proportion of NaNs greater than or equal to each threshold.


    """
    if thresholds is None:
        thresholds = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10]
    na_frac_rows = df.isna().mean(axis=1)
    return {thr: int((na_frac_rows >= thr).sum()) for thr in thresholds}

# Usage:
counts_rows = count_nan_rows_by_threshold(compustat)
print(counts_rows)


{0.95: 491, 0.9: 563807, 0.85: 602487, 0.8: 819957, 0.75: 955713, 0.7: 1143415, 0.6: 1524558, 0.5: 1925806, 0.4: 2040011, 0.3: 2051941, 0.2: 2052014, 0.1: 2052014}


In [ ]:
na_frac_rows = compustat.isna().mean(axis=1)
# Look at the 10 rows with the highest NaN fraction:
worst10 = na_frac_rows.sort_values(ascending=False).head(10).index
print(compustat.loc[worst10, compustat.columns[compustat.iloc[0].isna()]].head(10))

#Still have a lot of rows with empty nans.. lots of collumn may be really industry specific ... problematic

         acchgy  afudccy  afudciy  amcy  aolochy  apalchy  aqay  aqcy  aqdy  \
1933056     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1933057     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1933043     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1933044     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1933045     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1993889     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1993891     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1933046     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1933047     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   
1933048     NaN      NaN      NaN   NaN      NaN      NaN   NaN   NaN   NaN   

         aqepsy  ...  xopry  xoptdqpy  xoptdy  xoptepsqpy  xoptepsy  xoptqpy  \
1933056     NaN  ...    NaN       NaN     NaN     

In [ ]:
#Check how many duplicate pairs there are
duplicate_counts = (
    compustat
    .groupby(['gvkey', 'datadate'])
    .size()
    .reset_index(name='count')
)

# Keep only those with count > 1
duplicate_pairs = duplicate_counts[duplicate_counts['count'] > 1]

print(duplicate_pairs)

          gvkey    datadate  count
4434       1076  1995-03-31      2
6023       1098  1968-12-31      2
6238       1103  1966-12-31      2
7279       1117  1990-03-31      2
7750       1121  1979-03-31      2
...         ...         ...    ...
2046198  290862  2020-06-30      2
2048412  317427  2015-06-30      2
2048413  317427  2015-09-30      2
2049596  330227  2019-03-31      2
2049597  330227  2019-06-30      2

[1554 rows x 3 columns]


In [ ]:
#How much collumn and wich one differ among duplicated ( gvkey, datadate)

# 1) Find all rows where (gvkey, datadate) is duplicated
dup_rows = compustat[compustat.duplicated(subset=['gvkey', 'datadate'], keep=False)]

# 2) Group those rows by (gvkey, datadate)
grouped = dup_rows.groupby(['gvkey', 'datadate'])

# 3) Build a summary listing, for each duplicate group, which columns differ
diff_list = []
for (gvkey, datadate), grp in grouped:
    # Count distinct values per column within this group
    nunique_per_col = grp.nunique()
    # Keep columns where there is more than 1 unique value
    cols_that_differ = nunique_per_col[nunique_per_col > 1].index.tolist()
    diff_list.append({
        'gvkey': gvkey,
        'datadate': datadate,
        'num_columns_differ': len(cols_that_differ),
        'columns_that_differ': cols_that_differ
    })

# 4) Turn that into a DataFrame
diff_summary = pd.DataFrame(diff_list)

# 5) Display the summary
print(diff_summary.head(10))


   gvkey    datadate  num_columns_differ  \
0   1076  1995-03-31                  11   
1   1098  1968-12-31                   4   
2   1103  1966-12-31                   4   
3   1117  1990-03-31                   4   
4   1121  1979-03-31                  10   
5   1161  1987-03-31                  10   
6   1224  2013-12-31                  29   
7   1224  2014-03-31                  14   
8   1224  2014-06-30                  14   
9   1255  1988-03-31                   9   

                                 columns_that_differ  
0  [fyearq, fqtr, fyr, datafqtr, cshpry, epsfiy, ...  
1                      [fyearq, fqtr, fyr, datafqtr]  
2                      [fyearq, fqtr, fyr, datafqtr]  
3                      [fyearq, fqtr, fyr, datafqtr]  
4  [fyearq, fqtr, fyr, datafqtr, cshpry, epsfiy, ...  
5  [fyearq, fqtr, fyr, datafqtr, cshpry, epsfiy, ...  
6  [fyearq, fqtr, fyr, datafqtr, cibegniy, citota...  
7  [fyr, cibegniy, citotaly, ciy, dpy, ibadjy, ib...  
8  [fyr, cibegniy, c

  column pandas_dtype                                    sample_types
     tic       object                                 [<class 'str'>]
   cusip       object [<class 'str'>, <class 'int'>, <class 'float'>]
    conm       object                                 [<class 'str'>]
  curcdq       object                                 [<class 'str'>]
datacqtr       object                                 [<class 'str'>]
datafqtr       object                                 [<class 'str'>]
  costat       object                                 [<class 'str'>]
     fic       object                                 [<class 'str'>]


datadate:

tic:

cusip:

conm:

curcdq:

datacqtr:

datafqtr

costat:

fic: